# ANALISIS DEL PIB DEL LADO DE LA OFERTA
## PUNTOS A ANÁLIZAR
 1. - Participación del PIB por sector en el último y penultimo trimestre. Representar a través de un gráfico de torta.
 2. - Variacion Interanual del ultimo trimestre por sector.
 3. - Variacion Acumulada del ultimo trimestre por sector.

##### CONFIGURACIONES NECESARIAS

In [5]:
#Para situarnos en el directorio raiz del proyecto y correr adecuadamente el codigo con sus importaciones
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))


##### IMPORTACION DE LOS DATOS DEL CUADRO 6 

In [6]:
#Para recargar el modulo (cuando se tengan nuevos datos) se requiere de reload.
import importlib
import pandas as pd
import db.data_extraction as data_extraction

importlib.reload(data_extraction)
from db.data_extraction import select_cuadro_6
df_cuadro_6 = select_cuadro_6()



          valor       fecha nombre_actividad nombre_compuesto
0  1.097958e+06  1994-03-31      Agricultura             None
1  1.049813e+06  1994-06-30      Agricultura             None
2  9.771593e+05  1994-09-30      Agricultura             None
3  1.160269e+06  1994-12-31      Agricultura             None
4  1.385743e+06  1995-03-31      Agricultura             None
          valor       fecha nombre_actividad nombre_compuesto
0  1.097958e+06  1994-03-31      Agricultura             None
1  1.049813e+06  1994-06-30      Agricultura             None
2  9.771593e+05  1994-09-30      Agricultura             None
3  1.160269e+06  1994-12-31      Agricultura             None
4  1.385743e+06  1995-03-31      Agricultura             None


##### TRANSFORMACION DEL DF: Crear una única columna para clasificar las actividades

In [7]:
df_cuadro_6['Actividad_Economica'] = df_cuadro_6['nombre_actividad'].fillna(df_cuadro_6['nombre_compuesto'])

# 1. PARTICIPACION POR SECTOR

#### LOGICA DEL CALCULO

In [ ]:
#Filtrar el dataframe para extraer datos del ultimo trimestre
ultimo_trimestre = df_cuadro_6['fecha'].max() 
df_ultimo_trimestre = df_cuadro_6[df_cuadro_6['fecha'].isin([ultimo_trimestre])] 

#Extraer el valor del PIB del ultimo trimestre para usarlo como denominador
pib_total = df_ultimo_trimestre[df_ultimo_trimestre['Actividad_Economica'] =='PIB a precios de comprador']['valor'].values[0]

#Excluir el PIB y VAB de Actividad Economica (para el calculo del coeficinete de participacion)
df_actividades = df_ultimo_trimestre[(~df_ultimo_trimestre['Actividad_Economica'].isin(['PIB a precios de comprador', 'Valor agregado bruto']))].copy()

#Calcular la participacion de las actividades en el PIB 
df_actividades['Participacion'] = (df_actividades['valor']/pib_total)*100

print(f'PARTICIPACION DEL ULTIMO TRIMESTRE DISPONIBLE ({ultimo_trimestre}) EN EL PIB DEL TRIMESTRE: ')
print(df_actividades[['Actividad_Economica','Participacion']])

PARTICIPACION DEL ULTIMO TRIMESTRE DISPONIBLE (2024-12-31 00:00:00) EN EL PIB DEL TRIMESTRE: 
                      Actividad_Economica  Participacion
123                           Agricultura       5.536017
247  Ganaderia, forestal, pesca y mineria       3.115990
371                           Manufactura      19.548502
495                   Electricidad y agua       6.915275
619                          Construccion       6.139131
743                             Servicios      50.643660
991             Impuestos a los productos       8.101425


###### DATOS DEL DATAFRAME:

In [ ]:
etiquetas=df_ultimo_trimestre['Actividad_Economica'].unique() #identificamos el nombre concreto de las actividades 
print(etiquetas)

['Agricultura' 'Ganaderia, forestal, pesca y mineria' 'Manufactura'
 'Electricidad y agua' 'Construccion' 'Servicios' 'Valor agregado bruto'
 'Impuestos a los productos' 'PIB a precios de comprador']


# 2. VARIACION INTERANUAL POR SECTOR


In [ ]:
#Filtrar los datos del penultimo trimestre
anio_anterior = ultimo_trimestre - pd.DateOffset(years=1)
df_penultimo_trimestre = df_cuadro_6[df_cuadro_6['fecha'].isin([anio_anterior])]

#Transformar los datos con merge para unir la variacion del ultimo y penultimo periodo por Actividad
df_variacion_interanual = df_ultimo_trimestre.merge(df_penultimo_trimestre, on='Actividad_Economica',suffixes=('_actual','_anterior'))

#Calcular la variacion interanual por trimestre 
df_variacion_interanual['Variacion_interanual']= ((df_variacion_interanual['valor_actual']-df_variacion_interanual['valor_anterior'])/df_variacion_interanual['valor_anterior'])*100

print(f'Variacion interanual del ultimo trimestre disponible: ({ultimo_trimestre})')
print(df_variacion_interanual[['Actividad_Economica', 'Variacion_interanual']])

Variacion interanual del ultimo trimestre disponible: (2024-12-31 00:00:00) 
                    Actividad_Economica  Variacion_interanual
0                           Agricultura             -4.254609
1  Ganaderia, forestal, pesca y mineria              2.679270
2                           Manufactura              3.142439
3                   Electricidad y agua             -7.917132
4                          Construccion             13.635273
5                             Servicios              4.750939
6                  Valor agregado bruto              3.223560
7             Impuestos a los productos              8.276602
8            PIB a precios de comprador              3.615305


# 3. VARIACION ACUMULADA POR SECTOR

In [38]:
#Filtrar los datos del ultimo anio
anio_actual = ultimo_trimestre.year
df_anio_actual =df_cuadro_6[df_cuadro_6['fecha'].dt.year.isin([anio_actual])]

#Filtrar los datos del penultimo anio
df_penultimo_anio = df_cuadro_6[df_cuadro_6['fecha'].dt.year.isin([anio_actual-1])] 

#Tranformar los datos con groupbay para trabajar con suma de actividad por anio
df_acumulado_actual = df_anio_actual.groupby('Actividad_Economica')['valor'].sum().reset_index(name='valor_acumulado_actual')
df_acumulado_anterior = df_penultimo_anio.groupby('Actividad_Economica')['valor'].sum().reset_index(name='valor_acumulado_anterior')

#Transformamos los nuevos datos calculados con un merge por actividad
variacion_acumulada = df_acumulado_actual.merge(df_acumulado_anterior, on= 'Actividad_Economica')

#Calculamos la variacion acumulada por actividad
variacion_acumulada['Variacion_Acumulada']=((variacion_acumulada['valor_acumulado_actual']-variacion_acumulada['valor_acumulado_anterior'])/variacion_acumulada['valor_acumulado_anterior'])*100


print(f'Variacion acumulda por actividad para el ultimo trimestre disponible: ({ultimo_trimestre})')
print(variacion_acumulada[['Actividad_Economica','Variacion_Acumulada']])

Variacion acumulda por actividad para el ultimo trimestre disponible: (2024-12-31 00:00:00)
                    Actividad_Economica  Variacion_Acumulada
0                           Agricultura             2.776631
1                          Construccion             5.265773
2                   Electricidad y agua            -6.161906
3  Ganaderia, forestal, pesca y mineria             6.386391
4             Impuestos a los productos            10.108001
5                           Manufactura             4.437338
6            PIB a precios de comprador             4.249960
7                             Servicios             4.801336
8                  Valor agregado bruto             3.768006
